In [ ]:
import spacy
import json
import pickle
from spacy.util import filter_spans
from spacy.tokens import DocBin
from tqdm import tqdm

In [ ]:
with open('Corona2.json','r') as f:
    data=json.load(f)

In [ ]:
train_data = []
for example in data['examples']:
  temp_dict = {}
  temp_dict['text'] = example['content']
  temp_dict['entities'] = []
  for annotation in example['annotations']:
    start = annotation['start']
    end = annotation['end']
    label = annotation['tag_name'].upper()
    temp_dict['entities'].append((start, end, label))
  train_data.append(temp_dict)
train_data

In [ ]:
nlp = spacy.blank('en')
doc_bin = DocBin()
for training_example in tqdm(train_data):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        print(label)
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)
doc_bin.to_disk("train.spacy")

In [ ]:
! python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy